# Recurrency Revenue/Margin Analysis Plan

## Major Assumptions
We have access to a Recurrency PostgreSQL database containing transactional or operational data. The analysis will focus on identifying opportunities for revenue growth and margin improvement, assuming the database contains relevant sales, pricing, cost, and customer data. We'll need to first understand the schema before determining the specific analytical approach.

## Plan
- [x] Connect to database and explore schema structure
  - [x] Identify tables related to revenue, pricing, costs, and margins
  - [x] Document key relationships and data availability
- [ ] Analyze current revenue and margin patterns
  - [ ] Calculate baseline metrics by relevant dimensions (product, customer, time)
  - [ ] Identify high and low performers
- [ ] Identify improvement opportunities
  - [ ] Analyze pricing optimization potential and cost reduction areas
  - [ ] Quantify revenue/margin impact of recommended changes
- [ ] Summarize findings and recommendations

In [1]:
# Connect to Recurrency PostgreSQL database
import os
import psycopg2
import pandas as pd

# Connection using environment variables
conn = psycopg2.connect(
    host=os.environ['RECURRENCY_HOST'],
    port=int(os.environ.get('RECURRENCY_PORT', 5432)),
    database=os.environ['RECURRENCY_DATABASE'],
    user=os.environ['RECURRENCY_USERNAME'],
    password=os.environ['RECURRENCY_PASSWORD']
)

# List all tables in the database
tables_query = """
SELECT table_schema, table_name 
FROM information_schema.tables 
WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY table_schema, table_name;
"""

tables_df = pd.read_sql(tables_query, conn)
print(f"Found {len(tables_df)} tables:")
print(tables_df.to_string())

Found 2 tables:
  table_schema            table_name
0       public  recurrency_customers
1       public     recurrency_orders


/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_65134/1295850442.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql(tables_query, conn)


In [2]:
# Explore the structure of both tables
customers_schema = pd.read_sql("""
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_name = 'recurrency_customers'
ORDER BY ordinal_position;
""", conn)

orders_schema = pd.read_sql("""
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_name = 'recurrency_orders'
ORDER BY ordinal_position;
""", conn)

print("=== recurrency_customers columns ===")
print(customers_schema.to_string())
print("\n=== recurrency_orders columns ===")
print(orders_schema.to_string())

=== recurrency_customers columns ===
      column_name          data_type is_nullable
0     customer_id  character varying         YES
1    company_name  character varying         YES
2        industry  character varying         YES
3    go_live_date  character varying         YES
4       is_active            boolean         YES
5       plan_tier  character varying         YES
6  employee_count           smallint         YES

=== recurrency_orders columns ===
        column_name          data_type is_nullable
0          order_id  character varying         YES
1       customer_id  character varying         YES
2        order_date  character varying         YES
3      total_amount            numeric         YES
4      gross_profit            numeric         YES
5      order_status  character varying         YES
6  product_category  character varying         YES
7          quantity           smallint         YES


/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_65134/4283335492.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers_schema = pd.read_sql("""
/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_65134/4283335492.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders_schema = pd.read_sql("""


In [3]:
# Sample data from both tables to understand the data patterns
customers_sample = pd.read_sql("SELECT * FROM recurrency_customers LIMIT 10", conn)
orders_sample = pd.read_sql("SELECT * FROM recurrency_orders LIMIT 20", conn)

print("=== Customers Sample ===")
print(customers_sample.to_string())
print(f"\n=== Orders Sample ===")
print(orders_sample.to_string())

=== Customers Sample ===
  customer_id          company_name       industry go_live_date  is_active   plan_tier  employee_count
0    CUST-001    Acme Manufacturing  Manufacturing   2024-03-01       True     starter              50
1    CUST-002     Midwest Wholesale      Wholesale   2024-03-07       True      growth             100
2    CUST-003  Pacific Distribution         Retail   2024-03-13       True  enterprise             250
3    CUST-004      Delta Industrial   Distribution   2024-03-19       True     starter             500
4    CUST-005      Summit Supply Co     Industrial   2024-03-25       True      growth            1000
5    CUST-006      Harbor Logistics     Healthcare   2024-03-31       True  enterprise              50
6    CUST-007      Prime Components     Technology   2024-04-06       True     starter             100
7    CUST-008       Atlas Materials  Manufacturing   2024-04-12       True      growth             250
8    CUST-009   Vertex Distribution      Wholesa

/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_65134/566684107.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers_sample = pd.read_sql("SELECT * FROM recurrency_customers LIMIT 10", conn)
/var/folders/yj/2fhp0frx3z5ct5ktb9z_43l40000gn/T/ipykernel_65134/566684107.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders_sample = pd.read_sql("SELECT * FROM recurrency_orders LIMIT 20", conn)


In [4]:
# Load all data and calculate before vs after go-live metrics
import warnings
warnings.filterwarnings('ignore')

# Get all customers and orders
customers = pd.read_sql("SELECT * FROM recurrency_customers", conn)
orders = pd.read_sql("SELECT * FROM recurrency_orders", conn)

# Standardize date formats
customers['go_live_date'] = pd.to_datetime(customers['go_live_date'], format='mixed')
orders['order_date'] = pd.to_datetime(orders['order_date'], format='mixed')

# Merge orders with customer go_live_date
merged = orders.merge(customers[['customer_id', 'go_live_date', 'plan_tier', 'industry']], on='customer_id')

# Flag orders as before or after go-live
merged['period'] = merged.apply(lambda x: 'after' if x['order_date'] >= x['go_live_date'] else 'before', axis=1)

# Calculate margin percentage
merged['margin_pct'] = (merged['gross_profit'] / merged['total_amount']) * 100

print(f"Total orders: {len(merged):,}")
print(f"Orders before go-live: {(merged['period'] == 'before').sum():,}")
print(f"Orders after go-live: {(merged['period'] == 'after').sum():,}")
print(f"\nDate range: {merged['order_date'].min().date()} to {merged['order_date'].max().date()}")

Total orders: 35,695
Orders before go-live: 12,064
Orders after go-live: 23,631

Date range: 2023-06-01 to 2026-01-31


In [5]:
# Calculate per-customer metrics before and after go-live
customer_metrics = merged.groupby(['customer_id', 'period']).agg({
    'total_amount': ['sum', 'mean', 'count'],
    'gross_profit': ['sum', 'mean'],
    'margin_pct': 'mean'
}).reset_index()

customer_metrics.columns = ['customer_id', 'period', 'total_revenue', 'avg_order_value', 
                            'order_count', 'total_profit', 'avg_profit', 'avg_margin_pct']

# Pivot to compare before vs after
before = customer_metrics[customer_metrics['period'] == 'before'].set_index('customer_id')
after = customer_metrics[customer_metrics['period'] == 'after'].set_index('customer_id')

# Only include customers with BOTH before and after data for fair comparison
common_customers = before.index.intersection(after.index)
print(f"Customers with both before & after data: {len(common_customers)}")

# Create comparison dataframe
comparison = pd.DataFrame({
    'before_revenue': before.loc[common_customers, 'total_revenue'],
    'after_revenue': after.loc[common_customers, 'total_revenue'],
    'before_avg_order': before.loc[common_customers, 'avg_order_value'],
    'after_avg_order': after.loc[common_customers, 'avg_order_value'],
    'before_margin_pct': before.loc[common_customers, 'avg_margin_pct'],
    'after_margin_pct': after.loc[common_customers, 'avg_margin_pct'],
    'before_orders': before.loc[common_customers, 'order_count'],
    'after_orders': after.loc[common_customers, 'order_count']
})

# Calculate improvements
comparison['revenue_change_pct'] = ((comparison['after_revenue'] / comparison['before_revenue']) - 1) * 100
comparison['avg_order_change_pct'] = ((comparison['after_avg_order'] / comparison['before_avg_order']) - 1) * 100
comparison['margin_change_pp'] = comparison['after_margin_pct'] - comparison['before_margin_pct']  # percentage points
comparison['order_freq_change_pct'] = ((comparison['after_orders'] / comparison['before_orders']) - 1) * 100

comparison.head()

Customers with both before & after data: 30


,before_revenue,after_revenue,before_avg_order,after_avg_order,before_margin_pct,after_margin_pct,before_orders,after_orders,revenue_change_pct,avg_order_change_pct,margin_change_pp,order_freq_change_pct
customer_id,,,,,,,,,,,,
CUST-001,645919.97,2112748.92,2145.913522,2376.545467,22.997795,23.961144,301,889,227.091438,10.747495,0.963349,195.348837
CUST-002,662528.44,2155946.15,2151.066364,2447.157946,23.062195,25.547609,308,881,225.411865,13.764874,2.485414,186.038961
CUST-003,676715.66,2107546.35,2155.145414,2411.380263,23.011219,24.327709,314,874,211.437502,11.889446,1.316490,178.343949
CUST-004,693548.73,2104149.40,2160.587944,2429.733718,22.939566,25.955591,321,866,203.388833,12.457062,3.016025,169.781931
CUST-005,701123.65,2028900.21,2144.109021,2361.932724,23.059440,23.476235,327,859,189.378373,10.159171,0.416795,162.691131


In [6]:
# Calculate average improvement metrics across all customers
print("=" * 60)
print("   RECURRENCY AVERAGE IMPROVEMENT DELIVERED")
print("=" * 60)

avg_improvements = {
    'Avg Order Value': comparison['avg_order_change_pct'].mean(),
    'Margin (pp)': comparison['margin_change_pp'].mean(),
    'Order Frequency': comparison['order_freq_change_pct'].mean()
}

print(f"\n📈 Average Order Value Improvement:  +{avg_improvements['Avg Order Value']:.1f}%")
print(f"💰 Average Margin Improvement:       +{avg_improvements['Margin (pp)']:.2f} percentage points")
print(f"🔄 Average Order Frequency Increase: +{avg_improvements['Order Frequency']:.1f}%")

# Additional context
print(f"\n--- Supporting Metrics ---")
print(f"Before go-live avg margin: {comparison['before_margin_pct'].mean():.1f}%")
print(f"After go-live avg margin:  {comparison['after_margin_pct'].mean():.1f}%")
print(f"Before go-live avg order:  ${comparison['before_avg_order'].mean():,.0f}")
print(f"After go-live avg order:   ${comparison['after_avg_order'].mean():,.0f}")

   RECURRENCY AVERAGE IMPROVEMENT DELIVERED

📈 Average Order Value Improvement:  +12.7%
💰 Average Margin Improvement:       +1.95 percentage points
🔄 Average Order Frequency Increase: +102.6%

--- Supporting Metrics ---
Before go-live avg margin: 23.0%
After go-live avg margin:  25.0%
Before go-live avg order:  $2,150
After go-live avg order:   $2,423
